In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import seaborn as sns
import datetime, tqdm
import os
import matplotlib.patches as patches
pd.set_option('max_columns', 100)
#from kaggle.competitions import nflrush
from sklearn.model_selection import KFold, RepeatedKFold,GroupKFold
import math
import scipy
import numpy as np
import pandas as pd
import sklearn.metrics as mtr 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout, PReLU, BatchNormalization, ELU, GaussianNoise, Activation
from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping
from sklearn import preprocessing
import gc
import os
from tqdm import tqdm_notebook
import keras.backend as K

In [10]:
train_df = pd.read_csv(r'C:\Users\38980\OneDrive\Desktop\study\kaggle\NFL\train.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
#team abbreviations correct
def data_clean(df):
#correct name   
    df.loc[df['PossessionTeam'] == 'ARZ', 'PossessionTeam'] = 'ARI'
    df.loc[df['PossessionTeam'] == 'BLT', 'PossessionTeam'] = 'BAL'
    df.loc[df['PossessionTeam'] == 'CLV', 'PossessionTeam'] = 'CLE'
    df.loc[df['PossessionTeam'] == 'HST', 'PossessionTeam'] = 'HOU'
    df.loc[df['FieldPosition'] == 'ARZ', 'FieldPosition'] = 'ARI'
    df.loc[df['FieldPosition'] == 'BLT', 'FieldPosition'] = 'BAL'
    df.loc[df['FieldPosition'] == 'CLV', 'FieldPosition'] = 'CLE'
    df.loc[df['FieldPosition'] == 'HST', 'FieldPosition'] = 'HOU'

# fill null
    df = df.fillna(df.median())

# offense time and defence time
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['IsOnOffense'] = df.Team == df.TeamOnOffense # Is player on offense?
    
#time
    df['TimeHandoff'] = df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeSnap'] = df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['PlayerBirthDate'] = df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    seconds_in_year = 60*60*24*365.25
    df['PlayerAge'] = df.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
    df['GameClock'] = df['GameClock'].apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60)
    df['TimeDelta'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
#player height
    df['PlayerHeight'] = df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))

#weather
    def map_weather(txt):
        ans = 1
        if pd.isna(txt):
            return 0
        if 'partly' in txt:
            ans*=0.5
        if 'climate controlled' in txt or 'indoor' in txt:
            return ans*3
        if 'sunny' in txt or 'sun' in txt:
            return ans*2
        if 'clear' in txt:
            return ans
        if 'cloudy' in txt:
            return -ans
        if 'rain' in txt or 'rainy' in txt:
            return -2*ans
        if 'snow' in txt:
            return -3*ans
        return 0
    df['Cleaned_GameWeather'] = df['GameWeather'].str.lower()
    indoor = "indoor"
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['Cleaned_GameWeather'].apply(map_weather)

#diff Score    
    df["DiffScoreBeforePlay_ob"] = (df["HomeScoreBeforePlay"] - df["VisitorScoreBeforePlay"])
    df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]] = - df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]]
#Turf
    def agrupar_gramado(Turf):
        if Turf == 'Artifical':
            return 'Artificial'

        elif Turf in ('FieldTurf', 'Field turf'):
            return 'Field Turf'

        elif Turf in ('FieldTurf360', 'FieldTurf 360'):
            return 'Field Turf 360'

        elif Turf in ('Natural', 'Natural grass', 'Naturall Grass', 'grass', 'natural grass', 'SISGrass', 'Natural Grass'):
            return "Grass"

        elif Turf == "UBU Sports Speed S5-M":
            return "UBU Speed Series-S5-M"

        else:
            return Turf
    df['Turf'] = df['Turf'].apply(agrupar_gramado)
#left to right
    df['New_X'] = df['X']
    df.loc[df['PlayDirection'] == 'left','New_X'] = 120 - df.loc[df['PlayDirection'] == 'left','X']
    df['New_Y'] = df['Y']
    df.loc[df['PlayDirection'] == 'left','New_Y'] = 160/3 - df.loc[df['PlayDirection'] == 'left','Y']
    df['Orientation_std'] = df['Orientation']
    df.loc[df['Season'] == 2017, 'Orientation_std'] = df.loc[df['Season'] == 2017, 'Orientation_std'] + 90
    #df.loc[(df['Season'] > 2017)&(df['PlayDirection'] == 'left'), 'Orientation_std'] = 360 - df.loc[(df['Season'] > 2017)&(df['PlayDirection'] == 'left'), 'Orientation_std']
    df.loc[df['PlayDirection'] == 'left', 'Orientation_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Orientation_std'], 360)
    df['Dir_std'] = df['Dir']
    #df.loc[df['PlayDirection'] == 'left', 'Dir_std'] = 360 - df.loc[df['PlayDirection'] == 'left', 'Dir_std']
    df.loc[df['PlayDirection'] == 'left', 'Dir_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Dir_std'], 360)
    df['YardLine_std'] = 100 - df['YardLine']
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine_std'
         ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine']
    df["Orientation_sin"] = df["Orientation_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Orientation_cos"] = df["Orientation_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
    df["Dir_sin"] = df["Dir_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Dir_cos"] = df["Dir_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))

#distance and S
    #distance to yardline
    df['Dis_YardLine'] = df['New_X'] - df['YardLine_std'] - 10
    #distance to rusher
    def Distance(x1,x2,y1,y2):
        x_diff = (x1-x2)**2
        y_diff = (y1-y2)**2
        return np.sqrt(x_diff + y_diff)
    def Degree(x1,x2,y1,y2):
        try:
            tan = (y1-y2)/(x1-x2)
        except:
            tan = 0
        degree = 90 - math.atan(tan)/(2*np.pi)*360
        return degree
    df['IsRusher'] = (df['NflId'] == df['NflIdRusher'])
    Rusher =df.loc[df['IsRusher'],['PlayId','X','Y','Dir_std','S']].rename(columns={"X":"Rusher_X","Y":"Rusher_Y",'PossessionTeam':'Offense_Team','Dir_std':'Rusher_Dir_std','S':'Rusher_Speed'})
    df = df.merge(Rusher,how = 'left',on = 'PlayId')
    df['Distance_to_Rusher'] = df[["X","Rusher_X","Y","Rusher_Y"]].apply(lambda x: Distance(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_to_Rusher'] = df[["X","Rusher_X","Y","Rusher_Y"]].apply(lambda x: Degree(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_Diff'] = df['Degree_to_Rusher'] - df['Rusher_Dir_std']
    df['Speed_Ratio'] =  (df['Rusher_Speed']+0.01)/(df['S']+0.01)
    #back_direction
    def back_direction(orientation):
        if orientation > 180.0:
            return 1
        else:
            return 0
    df['back_oriented_down_field'] = df['Orientation_std'].apply(lambda x: back_direction(x))

# Stadium and location clean
    def agrupar_local(Location):
        if Location == "Arlington, Texas":
            return "Arlington, TX"
        elif Location in ("Baltimore, Maryland","Baltimore, Md."):
            return "Baltimore, MD"
        elif Location == "Charlotte, North Carolina":
            return "Charlotte, NC"
        elif Location == "Chicago. IL":
            return "Chicago, IL"
        elif Location == "Cincinnati, Ohio":
            return "Cincinnati, OH"
        elif Location in ("Cleveland","Cleveland Ohio","Cleveland, Ohio","Cleveland,Ohio"):
            return "Cleveland, OH"
        elif Location == "Detroit":
            return "Detroit, MI"
        elif Location == "E. Rutherford, NJ" or Location == "East Rutherford, N.J.":
            return "East Rutherford, NJ"
        elif Location == "Foxborough, Ma":
            return "Foxborough, MA"
        elif Location == "Houston, Texas":
            return "Houston, TX"
        elif Location in ("Jacksonville Florida","Jacksonville, Fl","Jacksonville, Florida"):
            return "Jacksonville, FL"
        elif Location == "London":
            return "London, England"
        elif Location == "Los Angeles, Calif.":
            return "Los Angeles, CA"
        elif Location == "Miami Gardens, Fla.":
            return "Miami Gardens, FLA"
        elif Location in ("New Orleans","New Orleans, La."):
            return "New Orleans, LA"
        elif Location == "Orchard Park NY":
            return "Orchard Park, NY"
        elif Location == "Philadelphia, Pa.":
            return "Philadelphia, PA"
        elif Location == "Pittsburgh":
            return "Pittsburgh, PA"
        elif Location == "Seattle":
            return "Seattle, WA"
        else:
            return Location

    df['Location'] = df['Location'].apply(agrupar_local)

# stadium types
    def agrupar_tipo_estadio(StadiumType):
        outdoor       = ['Outdoor', 'Outdoors', 'Cloudy', 'Heinz Field', 'Outdor', 'Ourdoor', 'Outside', 'Outddors', 'Outdoor Retr Roof-Open', 'Oudoor', 'Bowl']
        indoor_closed = ['Indoors', 'Indoor', 'Indoor, Roof Closed', 'Indoor, Roof Closed', 'Retractable Roof', 'Retr. Roof-Closed', 'Retr. Roof - Closed', 'Retr. Roof Closed']
        indoor_open   = ['Indoor, Open Roof', 'Open', 'Retr. Roof-Open', 'Retr. Roof - Open']
        dome_closed   = ['Dome', 'Domed, closed', 'Closed Dome', 'Domed', 'Dome, closed']
        dome_open     = ['Domed, Open', 'Domed, open']

        if StadiumType in outdoor:
            return 'outdoor'
        elif StadiumType in indoor_closed:
            return 'indoor_closed'
        elif StadiumType in indoor_open:
            return 'indoor_open'
        elif StadiumType in dome_closed:
            return 'dome_closed'
        elif StadiumType in dome_open:
            return 'dome_open'
        else:
            return 'unknown'
    df['StadiumType'] = df['StadiumType'].apply(agrupar_tipo_estadio)

# wind 
    def give_me_WindSpeed(x):
            x = str(x)
            x = x.replace('mph', '').strip()
            if '-' in x:
                x = (int(x.split('-')[0]) + int(x.split('-')[1])) / 2
            elif 'gusts up to' in x:
                x = (int(x.split()[0]) + int(x.split()[-1])) / 2
            elif 'clam' in x:
                x = 0
            try:
                return float(x)
            except:
                return -99
    df['Cleaned_WindSpeed'] = df['WindSpeed'].apply(give_me_WindSpeed)

# wind direction
    def agrupa_wind_direction(WindDirection):
        wd = str(WindDirection).upper()

        if wd == 'N' or 'FROM N' in wd:
            return 'north'
        if wd == 'S' or 'FROM S' in wd:
            return 'south'
        if wd == 'W' or 'FROM W' in wd:
            return 'west'
        if wd == 'E' or 'FROM E' in wd:
            return 'east'

        if 'FROM SW' in wd or 'FROM SSW' in wd or 'FROM WSW' in wd:
            return 'south west'
        if 'FROM SE' in wd or 'FROM SSE' in wd or 'FROM ESE' in wd:
            return 'south east'
        if 'FROM NW' in wd or 'FROM NNW' in wd or 'FROM WNW' in wd:
            return 'north west'
        if 'FROM NE' in wd or 'FROM NNE' in wd or 'FROM ENE' in wd:
            return 'north east'

        if 'NW' in wd or 'NORTHWEST' in wd:
            return 'north west'
        if 'NE' in wd or 'NORTH EAST' in wd:
            return 'north east'
        if 'SW' in wd or 'SOUTHWEST' in wd:
            return 'south west'
        if 'SE' in wd or 'SOUTHEAST' in wd:
            return 'south east'

        return 'unknown'
    
    df['WindDirection'] = df['WindDirection'].apply(agrupa_wind_direction)

# speed
    df.loc[df['Season'] == 2017, 'S'] = (df.loc[df['Season'] == 2017, 'S'] - 2.4355) / 1.2930 * 1.4551 + 2.7570
    df['Horizontal Speed'] = df['S']*df["Dir_sin"]
    df['Vertical Speed'] = df['S']*df["Dir_cos"]
# momentum
    df['momentum'] = df['S']*df['PlayerWeight']

#weight/ height
    df['weight_over_height'] = df['PlayerWeight']/df['PlayerHeight']**2

#2017?
    df['2017'] = df['Season'] == 2017
    
    return df

In [12]:
train = data_clean(train_df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in double_scalars


In [81]:
def split_data(df):
    df1 = df.loc[df['IsRusher']]
    df2 = df.loc[df['IsOnOffense'] & (~df['IsRusher'])]
    df3 = df.loc[~df['IsOnOffense']]

# max yards
    df1['Max_Yards'] = df1['YardLine']
    df1.loc[(df1.FieldPosition.fillna('') == df1.PossessionTeam)&(df1.PlayDirection =='left'),  'Max_Yards'] \
    = 100 - df1.loc[(df1.FieldPosition.fillna('') == df1.PossessionTeam)&(df1.PlayDirection =='left'),  'Max_Yards']
    df1.loc[(df1.FieldPosition.fillna('') != df1.PossessionTeam)&(df1.PlayDirection =='right'),  'Max_Yards'] \
    = 100 - df1.loc[(df1.FieldPosition.fillna('') != df1.PossessionTeam)&(df1.PlayDirection =='right'),  'Max_Yards']
# min_time_to_tacke
    df3['Min_Time_Tacke'] = (df3['Distance_to_Rusher']+0.01)/(df3['S']+0.01)

# defence_X_Y_spread
    Defence_X_Y_std = df3[["PlayId",'New_X','New_Y']].groupby("PlayId").std().rename(columns={'New_X':'Defense_X_std','New_Y':'Defense_Y_std'}) \
    .reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed2_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(0).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed2_std'}).reset_index()

    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_Y_Removed2_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(4).index, df3.groupby('PlayId').head(0).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed2_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed4_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(2).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed4_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_Y_Removed4_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(4).index, df3.groupby('PlayId').head(0).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Defence_X_Y_std, how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed4_std,how = 'left',  on ='PlayId')

#distance to QB
    dis_QB = df2.loc[df2["Position"] =='QB',['PlayId','Distance_to_Rusher']].groupby(['PlayId']).mean().rename(columns={'Distance_to_Rusher':'dis_to_QB'})
    df1 = df1.merge(dis_QB,how = 'left', on='PlayId')
#defence min,max,mean,std distance to rusher
    stat = df3.groupby(['GameId','PlayId']).agg({'Distance_to_Rusher':['min','max','mean','std']})
    stat.columns = stat.columns.droplevel()
    df1 = df1.merge(stat,how = 'left', on='PlayId')

# offense_X_Y_spread
    df2 = df2.sort_values(['PlayId','New_X'])
    Offense_X_Removed2_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed2_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed2_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed2_std'}).reset_index()
    
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_X_Removed4_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed4_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed4_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Offense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed4_std,how = 'left',  on ='PlayId')
    
# nearest offenders to defender
    dis_to_closest_offender = pd.DataFrame()
    for playid in df3['PlayId'].unique():
        offense = df2.loc[df2['PlayId'] == playid]
        defence = df3.loc[df3['PlayId'] == playid]
        ary = scipy.spatial.distance.cdist(defence[['New_X','New_Y']], offense[['New_X','New_Y']], metric='euclidean')
        ary.sort(axis=1)
        ary = pd.DataFrame(data = ary)
        ary['PlayId'] = playid
        ary.reset_index(drop=True, inplace=True)
        ary = pd.concat([ary, defence[['NflId']].reset_index(drop=True)], axis=1)
        dis_to_closest_offender = dis_to_closest_offender.append(ary)
    df3 = df3.merge(dis_to_closest_offender,how = 'left',on = ['PlayId','NflId'])
# distance to next defender
    def Distance(x1,x2,y1,y2):
        x_diff = (x1-x2)**2
        y_diff = (y1-y2)**2
        return np.sqrt(x_diff + y_diff)
    df3[['Next_X','Next_Y']] = df3[['New_X','New_Y']].shift(periods=-1,fill_value=0)
    df3['distance_to_next'] = df3[['Next_X','New_X','Next_Y','New_Y']].apply(lambda x: Distance(x[0],x[1],x[2],x[3]), axis = 1)
# personnel_features
    def defense_formation(l):
        dl = 0
        lb = 0
        db = 0
        other = 0

        for position in l:
            sub_string = position.split(' ')
            if sub_string[1] == 'DL':
                dl += int(sub_string[0])
            elif sub_string[1] in ['LB','OL']:
                lb += int(sub_string[0])
            else:
                db += int(sub_string[0])

        counts = (dl,lb,db,other)

        return counts
    def offense_formation(l):
        qb = 0
        rb = 0
        wr = 0
        te = 0
        ol = 0

        sub_total = 0
        qb_listed = False
        for position in l:
            sub_string = position.split(' ')
            pos = sub_string[1]
            cnt = int(sub_string[0])

            if pos == 'QB':
                qb += cnt
                sub_total += cnt
                qb_listed = True
            # Assuming LB is a line backer lined up as full back
            elif pos in ['RB','LB']:
                rb += cnt
                sub_total += cnt
            # Assuming DB is a defensive back and lined up as WR
            elif pos in ['WR','DB']:
                wr += cnt
                sub_total += cnt
            elif pos == 'TE':
                te += cnt
                sub_total += cnt
            # Assuming DL is a defensive lineman lined up as an additional line man
            else:
                ol += cnt
                sub_total += cnt

        # If not all 11 players were noted at given positions we need to make some assumptions
        # I will assume if a QB is not listed then there was 1 QB on the play
        # If a QB is listed then I'm going to assume the rest of the positions are at OL
        # This might be flawed but it looks like RB, TE and WR are always listed in the personnel
        if sub_total < 11:
            diff = 11 - sub_total
            if not qb_listed:
                qb += 1
                diff -= 1
            ol += diff

        counts = (qb,rb,wr,te,ol)

        return counts
    def split_personnel(s):
        splits = s.split(',')
        for i in range(len(splits)):
            splits[i] = splits[i].strip()

        return splits    
    def personnel_features(df):
        personnel = df[['GameId','PlayId','OffensePersonnel','DefensePersonnel']].drop_duplicates()
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: split_personnel(x))
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: defense_formation(x))
        personnel['num_DL'] = personnel['DefensePersonnel'].apply(lambda x: x[0])
        personnel['num_LB'] = personnel['DefensePersonnel'].apply(lambda x: x[1])
        personnel['num_DB'] = personnel['DefensePersonnel'].apply(lambda x: x[2])

        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: split_personnel(x))
        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: offense_formation(x))
        personnel['num_QB'] = personnel['OffensePersonnel'].apply(lambda x: x[0])
        personnel['num_RB'] = personnel['OffensePersonnel'].apply(lambda x: x[1])
        personnel['num_WR'] = personnel['OffensePersonnel'].apply(lambda x: x[2])
        personnel['num_TE'] = personnel['OffensePersonnel'].apply(lambda x: x[3])
        personnel['num_OL'] = personnel['OffensePersonnel'].apply(lambda x: x[4])

        # Let's create some features to specify if the OL is covered
        personnel['OL_diff'] = personnel['num_OL'] - personnel['num_DL']
        personnel['OL_TE_diff'] = (personnel['num_OL'] + personnel['num_TE']) - personnel['num_DL']
        # Let's create a feature to specify if the defense is preventing the run
        # Let's just assume 7 or more DL and LB is run prevention
        personnel['run_def'] = (personnel['num_DL'] + personnel['num_LB'] > 6).astype(int)

        personnel.drop(['OffensePersonnel','DefensePersonnel'], axis=1, inplace=True)
        
        return personnel
    
    personnel = personnel_features(df1)   
    df1 = df1.merge(personnel,how = 'left',  on ='PlayId')
    
#select useful columns    
    rusher = df1[['PlayId','TimeDelta','Team','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y', \
                 'Orientation_std','Dir_std','Dis_YardLine','Horizontal Speed','Vertical Speed','S','A','Dis','Position',\
                 'Quarter','GameClock','Down','Distance','OffenseFormation',\
                  'DefendersInTheBox','HomeScoreBeforePlay','VisitorScoreBeforePlay',\
                 'Offense_X_Removed2_std','Offense_Y_Removed2_std','Offense_X_Removed4_std','Offense_Y_Removed4_std',\
                 'Defense_X_std','Defense_Y_std','Defense_X_Removed2_std','Defense_Y_Removed2_std','Defense_X_Removed4_std',\
                 'Defense_Y_Removed4_std',\
                  'num_DL','num_LB','num_DB','num_QB','num_RB','num_WR','num_TE','num_OL','OL_diff','OL_TE_diff','run_def',\
                 'min','max','std','mean','dis_to_QB','Max_Yards','back_oriented_down_field',"DiffScoreBeforePlay_ob"]]
    rusher = rusher.sort_values('PlayId')
    game = df1[['PlayId','Turf']]
    game = game.sort_values('PlayId')
    offender = df2[['PlayId','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y','Orientation_std','Dir_std','back_oriented_down_field',\
                  'Horizontal Speed','Vertical Speed','S','A','Dis','Position','Distance_to_Rusher',\
                   'Degree_to_Rusher','Degree_Diff']]
    offender = offender.sort_values(['PlayId','Distance_to_Rusher'])
    defender = df3[['PlayId','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y','Orientation_std','Dir_std','back_oriented_down_field',\
                  'Horizontal Speed','Vertical Speed','S','A','Dis','Position','Distance_to_Rusher',\
                   'Degree_to_Rusher','Degree_Diff','Min_Time_Tacke','Speed_Ratio','distance_to_next']+list(range(2))] #list(range(2)) two closest offenders
    defender = defender.sort_values(['PlayId','Distance_to_Rusher'])

    
    return rusher, game, offender, defender

In [82]:
rusher, game, offender, defender = split_data(train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

In [15]:
#remove some columns
rusher = rusher.drop('max',axis = 1)
#game = game.drop(['Humidity'], axis = 1)

offender = offender.drop(['back_oriented_down_field'],axis = 1)
offender = offender.sort_values(['PlayId','Distance_to_Rusher']).groupby('PlayId').head(2)
defender = defender.sort_values(['PlayId','Distance_to_Rusher']).drop(['New_Y','back_oriented_down_field','Position'],axis = 1).groupby('PlayId').head(6)


In [16]:
#encoding fit
le = preprocessing.LabelEncoder()
categories =[]
for i in rusher.dtypes[rusher.dtypes=='object'].index.tolist():
    rusher[i] = rusher[i].astype(str)
    categories.append(rusher[i].unique())

for i in game.dtypes[game.dtypes=='object'].index.tolist():
    game[i] = game[i].astype(str)
    categories.append(game[i].unique())

for i in offender.dtypes[offender.dtypes=='object'].index.tolist():
    offender[i] = offender[i].astype(str)
    categories.append(offender[i].unique())

for i in defender.dtypes[defender.dtypes=='object'].index.tolist():
    defender[i] = defender[i].astype(str)
    categories.append(defender[i].unique())
categories = np.hstack(categories)
le.fit(categories)
le.classes_

array(['A-Turf Titan', 'ACE', 'Artificial', 'C', 'CB', 'DD GrassMaster',
       'DE', 'DT', 'EMPTY', 'FB', 'Field Turf', 'Field Turf 360', 'G',
       'Grass', 'HB', 'I_FORM', 'JUMBO', 'NT', 'OG', 'OLB', 'OT',
       'PISTOL', 'QB', 'RB', 'SHOTGUN', 'SINGLEBACK', 'T', 'TE',
       'Twenty-Four/Seven Turf', 'UBU Speed Series-S5-M', 'WILDCAT', 'WR',
       'away', 'home', 'nan'], dtype=object)

In [17]:
#encoding transform
for i in rusher.dtypes[rusher.dtypes=='object'].index.tolist():
    rusher[i] = le.transform(rusher[i])

for i in game.dtypes[game.dtypes=='object'].index.tolist():
    game[i] = le.transform(game[i])

for i in offender.dtypes[offender.dtypes=='object'].index.tolist():
    offender[i] = le.transform(offender[i])

for i in defender.dtypes[defender.dtypes=='object'].index.tolist():
    defender[i] = le.transform(defender[i])

In [523]:
#reshape offender
offender_players = [offender.drop('PlayId',axis = 1).iloc[np.arange(k, len(offender), 2)].reset_index(drop = True) for k in range(2)]
offender_players = np.hstack([t.values for t in offender_players])
offender_players.shape

(23171, 32)

In [524]:
#reshape defender
defender_players = [defender.drop('PlayId',axis = 1).iloc[np.arange(k, len(defender), 6)].reset_index(drop = True) for k in range(6)]
defender_players = np.hstack([t.values for t in defender_players])
defender_players.shape

(23171, 114)

In [525]:
#train
train_y =train.loc[train['IsRusher'],['Yards','PlayId']].sort_values('PlayId').drop('PlayId',axis = 1)
train_x = np.hstack([rusher.drop('PlayId',axis = 1).values,game.drop('PlayId',axis = 1).values,defender_players,offender_players])
train_y99 =(train_y + 99).reset_index(drop=True).Yards

In [526]:
#loss function
def crps_eval(y_pred, dataset, is_higher_better=False):
    labels = dataset.get_label()
    labels = labels.astype('int')
    y_true = np.zeros((len(labels),199))
    for i, v in enumerate(labels):
        y_true[i, v:] = 1
    y_pred = y_pred.reshape(-1, 199, order='F')
    y_pred = np.clip(y_pred.cumsum(axis=1), 0, 1)
    return 'crps', np.mean((y_pred - y_true)**2), False

In [531]:
# hyper parameters
params = {'max_depth':5, 'lambda_l1': 1.6, 'lambda_l2': 1.2,
 'num_leaves': 32, 'feature_fraction': 0.4,
 'subsample': 0.4, 'min_child_samples': 15,
 'learning_rate': 0.02,
 'num_iterations': 10000, 'random_state': 42,
 'objective': 'multiclass',
 'min_gain_to_split':0.9,
 'num_class':199,
 'metric':'None',
 'max_bin':200}


In [532]:
import lightgbm as lgb
models = []
for k in range(1):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_x)):
        tr_x, tr_y = train_x[tr_inds], train_y99[tr_inds]    
        vl_x, v_y = train_x[val_inds], train_y99[val_inds] 
        dtrain = lgb.Dataset(tr_x, label= tr_y)
        dvalid = lgb.Dataset(vl_x, label= v_y)
        model = lgb.train(params, dtrain,
                              num_boost_round=100000,
                              valid_sets=[dtrain,dvalid],
                              early_stopping_rounds=20,
                              verbose_eval=100,
                              feval=crps_eval)
        models.append(model)

Training until validation scores don't improve for 20 rounds
[100]	training's crps: 0.0127428	valid_1's crps: 0.0133483
[200]	training's crps: 0.0117391	valid_1's crps: 0.0130523
[300]	training's crps: 0.0111141	valid_1's crps: 0.012924
[400]	training's crps: 0.0107027	valid_1's crps: 0.0128641
[500]	training's crps: 0.0104175	valid_1's crps: 0.0128313
[600]	training's crps: 0.0102015	valid_1's crps: 0.0128089
[700]	training's crps: 0.0100485	valid_1's crps: 0.0127969
[800]	training's crps: 0.00993492	valid_1's crps: 0.0127884
[900]	training's crps: 0.00986698	valid_1's crps: 0.0127845
Early stopping, best iteration is:
[926]	training's crps: 0.0098556	valid_1's crps: 0.0127836
Training until validation scores don't improve for 20 rounds
[100]	training's crps: 0.0127819	valid_1's crps: 0.0132707
[200]	training's crps: 0.0117908	valid_1's crps: 0.012961
[300]	training's crps: 0.0111756	valid_1's crps: 0.012834
[400]	training's crps: 0.0107908	valid_1's crps: 0.0127734
[500]	training's c

KeyboardInterrupt: 

In [503]:
#CV score

# for leave out one feature importance compare
print(np.mean([0.0129045,0.0128677,0.0127771,0.0127333,0.013174]))

# all defense palyer
print(np.mean([0.012802,0.0127486,0.0126934,0.0126226,0.0130668]))

# 5 closest defender + 2 closest offender l1,l2 = 1.2
print(np.mean([0.0127848,0.0127063,0.0126001,0.0126058,0.013049]))

# 6 closest defender + 2 closest offender l1,l2 = 1.3, 1.2
print(np.mean([0.0127827,0.012679,0.0126066,0.0125891,0.0130469]))

# 6 closest defender + 2 closest offender l1,l2 = 1.3, 1.2
print(np.mean([0.0127806,0.0126752,0.0126092,0.0125885,0.0130475]))

# all defense palyer + 2 closest offender l1,l2 = 1.8  1.4
print(np.mean([0.0127711,0.012681,0.0126273,0.0125774,0.013032]))


0.012891320000000001
0.01278668
0.012749200000000002
0.012740859999999998
0.012740200000000002
0.01273776


In [93]:
X_Dense = rusher[['Dis_YardLine','PlayerAge','PlayerHeight','PlayerWeight', \
            'Orientation_std','Dir_std','New_X','New_Y','Horizontal Speed',\
            'Vertical Speed','S','A','Dis','GameClock',\
            'Distance','Max_Yards',\
            'Defense_X_std','Defense_Y_std','Defense_X_Removed2_std',\
            'Offense_Y_Removed2_std',\
           'min','std','mean','Max_Yards','dis_to_QB',\
                 'Quarter','Down','HomeScoreBeforePlay','VisitorScoreBeforePlay',\
                  "DiffScoreBeforePlay_ob"\
]]
X_Dense = X_Dense.fillna(0)
X_Cat = rusher[['Quarter','Down','Position','OffenseFormation']]

yards = train.loc[train['IsRusher'],['Yards','PlayId']].\
sort_values('PlayId').drop('PlayId',axis = 1).Yards

y = np.zeros((yards.shape[0], 199))
for idx, target in enumerate(list(yards)):
    y[idx][99 + target] = 1

In [94]:
X = X_Dense
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [95]:
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from keras.models import Model
import keras.backend as K
import re
from keras.losses import binary_crossentropy
from  keras.callbacks import EarlyStopping,ModelCheckpoint
import codecs

from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from sklearn.metrics import f1_score




class CRPSCallback(Callback):
    
    def __init__(self,validation, predict_batch_size=20, include_on_batch=False):
        super(CRPSCallback, self).__init__()
        self.validation = validation
        self.predict_batch_size = predict_batch_size
        self.include_on_batch = include_on_batch
        
        print('validation shape',len(self.validation))

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_train_begin(self, logs={}):
        if not ('CRPS_score_val' in self.params['metrics']):
            self.params['metrics'].append('CRPS_score_val')

    def on_batch_end(self, batch, logs={}):
        if (self.include_on_batch):
            logs['CRPS_score_val'] = float('-inf')

    def on_epoch_end(self, epoch, logs={}):
        logs['CRPS_score_val'] = float('-inf')
            
        if (self.validation):
            X_valid, y_valid = self.validation[0], self.validation[1]
            y_pred = self.model.predict(X_valid)
            y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
            y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
            val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
            val_s = np.round(val_s, 6)
            logs['CRPS_score_val'] = val_s

In [96]:
def get_model(x_tr,y_tr,x_val,y_val):
    inp = Input(shape = (x_tr.shape[1],))
    x = Dense(1024, input_dim=X.shape[1], activation='relu')(inp)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    
    out = Dense(199, activation='softmax')(x)
    model = Model(inp,out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[])
    #add lookahead
#     lookahead = Lookahead(k=5, alpha=0.5) # Initialize Lookahead
#     lookahead.inject(model) # add into model

    
    es = EarlyStopping(monitor='CRPS_score_val', 
                       mode='min',
                       restore_best_weights=True, 
                       verbose=1, 
                       patience=10)

    mc = ModelCheckpoint('best_model.h5',monitor='CRPS_score_val',mode='min',
                                   save_best_only=True, verbose=1, save_weights_only=True)
    
    bsz = 1024
    steps = x_tr.shape[0]/bsz
    


    model.fit(x_tr, y_tr,callbacks=[CRPSCallback(validation = (x_val,y_val)),es,mc], epochs=100, batch_size=bsz,verbose=1)
    model.load_weights("best_model.h5")
    
    y_pred = model.predict(x_val)
    y_valid = y_val
    y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * x_val.shape[0])
    crps = np.round(val_s, 6)

    return model,crps

In [97]:
from sklearn.model_selection import train_test_split, KFold
import time

losses = []
models = []
crps_csv = []

s_time = time.time()


for k in range(2):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(yards)):
        print("-----------")
        print("-----------")
        tr_x,tr_y = X[tr_inds],y[tr_inds]
        val_x,val_y = X[val_inds],y[val_inds]
        model,crps = get_model(tr_x,tr_y,val_x,val_y)
        models.append(model)
        print("the %d fold crps is %f"%((k_fold+1),crps))
        crps_csv.append(crps)
 
print("mean crps is %f"%np.mean(crps_csv))


def predict(x_te):
    model_num = len(models)
    for k,m in enumerate(models):
        if k==0:
            y_pred = m.predict(x_te,batch_size=1024)
        else:
            y_pred+=m.predict(x_te,batch_size=1024)
            
    y_pred = y_pred / model_num
    
    return y_pred

-----------
-----------
validation shape 2
Epoch 1/100
18536/18536 [==============================] - ETA: 12s - loss: 5.93 - ETA: 6s - loss: 5.8708 - ETA: 4s - loss: 5.859 - ETA: 3s - loss: 5.831 - ETA: 3s - loss: 5.798 - ETA: 2s - loss: 5.782 - ETA: 2s - loss: 5.761 - ETA: 2s - loss: 5.742 - ETA: 1s - loss: 5.731 - ETA: 1s - loss: 5.713 - ETA: 1s - loss: 5.698 - ETA: 1s - loss: 5.680 - ETA: 0s - loss: 5.667 - ETA: 0s - loss: 5.653 - ETA: 0s - loss: 5.643 - ETA: 0s - loss: 5.627 - ETA: 0s - loss: 5.617 - ETA: 0s - loss: 5.603 - 3s 164us/step - loss: 5.6030

Epoch 00001: CRPS_score_val improved from inf to 0.08250, saving model to best_model.h5
Epoch 2/100
18536/18536 [==============================] - ETA: 2s - loss: 5.338 - ETA: 2s - loss: 5.355 - ETA: 1s - loss: 5.310 - ETA: 1s - loss: 5.289 - ETA: 1s - loss: 5.275 - ETA: 1s - loss: 5.267 - ETA: 1s - loss: 5.259 - ETA: 1s - loss: 5.250 - ETA: 1s - loss: 5.244 - ETA: 1s - loss: 5.232 - ETA: 0s - loss: 5.224 - ETA: 0s - loss: 5.213 - 

18536/18536 [==============================] - ETA: 2s - loss: 2.842 - ETA: 1s - loss: 2.860 - ETA: 1s - loss: 2.848 - ETA: 1s - loss: 2.836 - ETA: 1s - loss: 2.823 - ETA: 1s - loss: 2.811 - ETA: 1s - loss: 2.806 - ETA: 1s - loss: 2.801 - ETA: 1s - loss: 2.802 - ETA: 1s - loss: 2.802 - ETA: 0s - loss: 2.794 - ETA: 0s - loss: 2.795 - ETA: 0s - loss: 2.792 - ETA: 0s - loss: 2.797 - ETA: 0s - loss: 2.797 - ETA: 0s - loss: 2.796 - ETA: 0s - loss: 2.798 - ETA: 0s - loss: 2.798 - 2s 124us/step - loss: 2.7994

Epoch 00014: CRPS_score_val improved from 0.01323 to 0.01318, saving model to best_model.h5
Epoch 15/100
18536/18536 [==============================] - ETA: 2s - loss: 2.734 - ETA: 1s - loss: 2.739 - ETA: 1s - loss: 2.765 - ETA: 1s - loss: 2.763 - ETA: 1s - loss: 2.755 - ETA: 1s - loss: 2.757 - ETA: 1s - loss: 2.758 - ETA: 1s - loss: 2.762 - ETA: 1s - loss: 2.766 - ETA: 1s - loss: 2.777 - ETA: 0s - loss: 2.777 - ETA: 0s - loss: 2.776 - ETA: 0s - loss: 2.778 - ETA: 0s - loss: 2.784 - ETA

18536/18536 [==============================] - ETA: 2s - loss: 2.684 - ETA: 2s - loss: 2.704 - ETA: 1s - loss: 2.718 - ETA: 1s - loss: 2.718 - ETA: 1s - loss: 2.702 - ETA: 1s - loss: 2.705 - ETA: 1s - loss: 2.688 - ETA: 1s - loss: 2.698 - ETA: 1s - loss: 2.701 - ETA: 1s - loss: 2.705 - ETA: 0s - loss: 2.705 - ETA: 0s - loss: 2.706 - ETA: 0s - loss: 2.706 - ETA: 0s - loss: 2.707 - ETA: 0s - loss: 2.709 - ETA: 0s - loss: 2.705 - ETA: 0s - loss: 2.706 - ETA: 0s - loss: 2.706 - 2s 126us/step - loss: 2.7070

Epoch 00027: CRPS_score_val improved from 0.01291 to 0.01291, saving model to best_model.h5
Epoch 28/100
18536/18536 [==============================] - ETA: 2s - loss: 2.716 - ETA: 2s - loss: 2.725 - ETA: 1s - loss: 2.721 - ETA: 1s - loss: 2.711 - ETA: 1s - loss: 2.710 - ETA: 1s - loss: 2.705 - ETA: 1s - loss: 2.711 - ETA: 1s - loss: 2.710 - ETA: 1s - loss: 2.709 - ETA: 1s - loss: 2.707 - ETA: 0s - loss: 2.709 - ETA: 0s - loss: 2.707 - ETA: 0s - loss: 2.708 - ETA: 0s - loss: 2.706 - ETA

18536/18536 [==============================] - ETA: 2s - loss: 2.664 - ETA: 2s - loss: 2.636 - ETA: 1s - loss: 2.636 - ETA: 1s - loss: 2.652 - ETA: 1s - loss: 2.666 - ETA: 1s - loss: 2.673 - ETA: 1s - loss: 2.676 - ETA: 1s - loss: 2.681 - ETA: 1s - loss: 2.682 - ETA: 1s - loss: 2.672 - ETA: 0s - loss: 2.674 - ETA: 0s - loss: 2.672 - ETA: 0s - loss: 2.680 - ETA: 0s - loss: 2.674 - ETA: 0s - loss: 2.671 - ETA: 0s - loss: 2.670 - ETA: 0s - loss: 2.666 - ETA: 0s - loss: 2.665 - 2s 126us/step - loss: 2.6662

Epoch 00040: CRPS_score_val improved from 0.01285 to 0.01284, saving model to best_model.h5
Epoch 41/100
18536/18536 [==============================] - ETA: 2s - loss: 2.706 - ETA: 2s - loss: 2.698 - ETA: 1s - loss: 2.672 - ETA: 1s - loss: 2.652 - ETA: 1s - loss: 2.662 - ETA: 1s - loss: 2.673 - ETA: 1s - loss: 2.680 - ETA: 1s - loss: 2.681 - ETA: 1s - loss: 2.681 - ETA: 1s - loss: 2.682 - ETA: 0s - loss: 2.682 - ETA: 0s - loss: 2.678 - ETA: 0s - loss: 2.675 - ETA: 0s - loss: 2.671 - ETA

18536/18536 [==============================] - ETA: 2s - loss: 2.622 - ETA: 2s - loss: 2.635 - ETA: 1s - loss: 2.629 - ETA: 1s - loss: 2.643 - ETA: 1s - loss: 2.648 - ETA: 1s - loss: 2.633 - ETA: 1s - loss: 2.629 - ETA: 1s - loss: 2.627 - ETA: 1s - loss: 2.634 - ETA: 1s - loss: 2.630 - ETA: 0s - loss: 2.629 - ETA: 0s - loss: 2.628 - ETA: 0s - loss: 2.632 - ETA: 0s - loss: 2.632 - ETA: 0s - loss: 2.633 - ETA: 0s - loss: 2.633 - ETA: 0s - loss: 2.633 - ETA: 0s - loss: 2.634 - 2s 125us/step - loss: 2.6346

Epoch 00053: CRPS_score_val did not improve from 0.01282
Epoch 54/100
18536/18536 [==============================] - ETA: 2s - loss: 2.651 - ETA: 2s - loss: 2.631 - ETA: 1s - loss: 2.640 - ETA: 1s - loss: 2.633 - ETA: 1s - loss: 2.637 - ETA: 1s - loss: 2.630 - ETA: 1s - loss: 2.629 - ETA: 1s - loss: 2.635 - ETA: 1s - loss: 2.636 - ETA: 1s - loss: 2.630 - ETA: 0s - loss: 2.632 - ETA: 0s - loss: 2.630 - ETA: 0s - loss: 2.630 - ETA: 0s - loss: 2.631 - ETA: 0s - loss: 2.630 - ETA: 0s - loss

...

18537/18537 [==============================] - ETA: 2s - loss: 2.806 - ETA: 1s - loss: 2.777 - ETA: 1s - loss: 2.762 - ETA: 1s - loss: 2.764 - ETA: 1s - loss: 2.765 - ETA: 1s - loss: 2.764 - ETA: 1s - loss: 2.770 - ETA: 1s - loss: 2.770 - ETA: 1s - loss: 2.769 - ETA: 1s - loss: 2.770 - ETA: 0s - loss: 2.770 - ETA: 0s - loss: 2.766 - ETA: 0s - loss: 2.756 - ETA: 0s - loss: 2.759 - ETA: 0s - loss: 2.762 - ETA: 0s - loss: 2.762 - ETA: 0s - loss: 2.760 - ETA: 0s - loss: 2.758 - 2s 122us/step - loss: 2.7585

Epoch 00018: CRPS_score_val improved from 0.01307 to 0.01303, saving model to best_model.h5
Epoch 19/100
18537/18537 [==============================] - ETA: 2s - loss: 2.726 - ETA: 1s - loss: 2.741 - ETA: 1s - loss: 2.761 - ETA: 1s - loss: 2.754 - ETA: 1s - loss: 2.748 - ETA: 1s - loss: 2.751 - ETA: 1s - loss: 2.755 - ETA: 1s - loss: 2.751 - ETA: 1s - loss: 2.745 - ETA: 1s - loss: 2.743 - ETA: 0s - loss: 2.749 - ETA: 0s - loss: 2.742 - ETA: 0s - loss: 2.744 - ETA: 0s - loss: 2.743 - ETA

18537/18537 [==============================] - ETA: 2s - loss: 2.670 - ETA: 2s - loss: 2.689 - ETA: 1s - loss: 2.668 - ETA: 1s - loss: 2.690 - ETA: 1s - loss: 2.687 - ETA: 1s - loss: 2.683 - ETA: 1s - loss: 2.686 - ETA: 1s - loss: 2.681 - ETA: 1s - loss: 2.676 - ETA: 1s - loss: 2.682 - ETA: 0s - loss: 2.679 - ETA: 0s - loss: 2.680 - ETA: 0s - loss: 2.678 - ETA: 0s - loss: 2.678 - ETA: 0s - loss: 2.682 - ETA: 0s - loss: 2.685 - ETA: 0s - loss: 2.685 - ETA: 0s - loss: 2.688 - 2s 126us/step - loss: 2.6882

Epoch 00031: CRPS_score_val improved from 0.01288 to 0.01287, saving model to best_model.h5
Epoch 32/100
18537/18537 [==============================] - ETA: 2s - loss: 2.722 - ETA: 2s - loss: 2.729 - ETA: 1s - loss: 2.708 - ETA: 1s - loss: 2.694 - ETA: 1s - loss: 2.695 - ETA: 1s - loss: 2.703 - ETA: 1s - loss: 2.700 - ETA: 1s - loss: 2.683 - ETA: 1s - loss: 2.689 - ETA: 1s - loss: 2.683 - ETA: 0s - loss: 2.678 - ETA: 0s - loss: 2.677 - ETA: 0s - loss: 2.677 - ETA: 0s - loss: 2.678 - ETA

Epoch 00044: CRPS_score_val did not improve from 0.01285
Epoch 45/100
18537/18537 [==============================] - ETA: 2s - loss: 2.657 - ETA: 2s - loss: 2.631 - ETA: 1s - loss: 2.656 - ETA: 1s - loss: 2.648 - ETA: 1s - loss: 2.638 - ETA: 1s - loss: 2.639 - ETA: 1s - loss: 2.638 - ETA: 1s - loss: 2.643 - ETA: 1s - loss: 2.638 - ETA: 0s - loss: 2.639 - ETA: 0s - loss: 2.644 - ETA: 0s - loss: 2.648 - ETA: 0s - loss: 2.644 - ETA: 0s - loss: 2.649 - ETA: 0s - loss: 2.648 - ETA: 0s - loss: 2.648 - ETA: 0s - loss: 2.647 - ETA: 0s - loss: 2.647 - 2s 122us/step - loss: 2.6468

Epoch 00045: CRPS_score_val did not improve from 0.01285
Epoch 46/100
18537/18537 [==============================] - ETA: 2s - loss: 2.662 - ETA: 1s - loss: 2.673 - ETA: 1s - loss: 2.658 - ETA: 1s - loss: 2.650 - ETA: 1s - loss: 2.650 - ETA: 1s - loss: 2.648 - ETA: 1s - loss: 2.649 - ETA: 1s - loss: 2.646 - ETA: 1s - loss: 2.647 - ETA: 1s - loss: 2.643 - ETA: 0s - loss: 2.636 - ETA: 0s - loss: 2.641 - ETA: 0s - loss: 


Epoch 00058: CRPS_score_val did not improve from 0.01284
Epoch 59/100
18537/18537 [==============================] - ETA: 2s - loss: 2.663 - ETA: 1s - loss: 2.633 - ETA: 1s - loss: 2.620 - ETA: 1s - loss: 2.624 - ETA: 1s - loss: 2.621 - ETA: 1s - loss: 2.608 - ETA: 1s - loss: 2.602 - ETA: 1s - loss: 2.602 - ETA: 1s - loss: 2.601 - ETA: 1s - loss: 2.599 - ETA: 0s - loss: 2.599 - ETA: 0s - loss: 2.603 - ETA: 0s - loss: 2.605 - ETA: 0s - loss: 2.608 - ETA: 0s - loss: 2.606 - ETA: 0s - loss: 2.606 - ETA: 0s - loss: 2.605 - ETA: 0s - loss: 2.605 - 2s 122us/step - loss: 2.6062

Epoch 00059: CRPS_score_val did not improve from 0.01284
Epoch 60/100
18537/18537 [==============================] - ETA: 2s - loss: 2.553 - ETA: 2s - loss: 2.561 - ETA: 1s - loss: 2.582 - ETA: 1s - loss: 2.596 - ETA: 1s - loss: 2.606 - ETA: 1s - loss: 2.607 - ETA: 1s - loss: 2.603 - ETA: 1s - loss: 2.603 - ETA: 1s - loss: 2.600 - ETA: 0s - loss: 2.605 - ETA: 0s - loss: 2.604 - ETA: 0s - loss: 2.607 - ETA: 0s - loss:

KeyboardInterrupt: 